In [ ]:
!pip install tensorflow==2.15.0 gdown

In [ ]:
# download the images folder
import gdown
id = "1Ki0ZV4LXLPDLsQ_knX-9c3gC2xCqbEMA"
gdown.download(id=id, output="data.zip")

In [ ]:
# Full data (image and masks)

# import gdown

# # Google Drive file ID (update this with the new ID of the zip file)
# file_id = "1OMkWH3TXdvFKhfUqVfC1sWfTArjJn3pO"

# # Download the zip file
# gdown.download(id=file_id, output="full_data.zip", quiet=False)

# # Optionally, you can unzip the downloaded file
# import zipfile
# with zipfile.ZipFile("full_data.zip", 'r') as zip_ref:
#     zip_ref.extractall("full_data")


In [ ]:
# Dowbnload the models folder
url = "https://drive.google.com/drive/folders/13zsFUu8JAZzx9tht0beD0KTNai7uZzpM?usp=drive_link"
gdown.download_folder(url)

In [ ]:
# Unzip the data files
import zipfile 

with zipfile.ZipFile("data.zip") as ref:
    ref.extractall()
    ref.close()
    

In [ ]:
# Setting up the parent directory as the working directory
import sys
sys.path.append('../')

import tensorflow as  tf
from keras.models import load_model
import os
from PIL import Image

from tqdm.auto import tqdm
from sklearn.model_selection import train_test_split


In [ ]:
import numpy as np
import tensorflow as tf
# from tensorflow.keras import backend as K
import keras.backend as K 

def iou(y_true, y_pred):
    def f(y_true, y_pred):
        intersection = (y_true * y_pred).sum()
        union = y_true.sum() + y_pred.sum() - intersection
        x = (intersection + 1e-15) / (union + 1e-15)
        x = x.astype(np.float32)
        return x
    return tf.numpy_function(f, [y_true, y_pred], tf.float32)

smooth = 1e-15
def dice_coef(y_true, y_pred):
    y_true = tf.keras.layers.Flatten()(y_true)
    y_pred = tf.keras.layers.Flatten()(y_pred)
    y_true = tf.cast(y_true, dtype=tf.float32)
    y_pred = tf.cast(y_pred, dtype=tf.float32)

    intersection = tf.reduce_sum(y_true * y_pred)
    return (2. * intersection + smooth) / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) + smooth)

def dice_loss(y_true, y_pred):
    return 1.0 - dice_coef(y_true, y_pred)

In [ ]:
# For new model
model = load_model("/kaggle/working/models/deeplabv3_updated.h5", custom_objects={'dice_coef': dice_coef,
                                                             'iou': iou})

In [ ]:
# import os
# import glob

# # Define the directory path
# directory_path = '/kaggle/working/images'

# # Use glob to find all image files with the specified extensions
# image_files = glob.glob(os.path.join(directory_path, '*.jpg')) + \
#               glob.glob(os.path.join(directory_path, '*.jpeg')) + \
#               glob.glob(os.path.join(directory_path, '*.png')) + \
#               glob.glob(os.path.join(directory_path, '*.gif')) + \
#               glob.glob(os.path.join(directory_path, '*.bmp'))

# # Count the number of image files
# total_images = len(image_files)

# print(f"Total number of images: {total_images}")

In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import expand_dims
from tensorflow.image import resize

# Path to the folder containing images
folder_path = "/kaggle/working/images"

# Function to load and preprocess images from a folder
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        if img is not None:
            # Resize to 256x256
            img = resize(img, (256, 256))
            # Normalize
            img = img / 255.0
            images.append(img)
    return images

# Load images from the folder
images = load_images_from_folder(folder_path)



# Iterate through images
for image in images:
    pred_mask = model.predict(expand_dims(image, axis=0))[0]
    post_process = (pred_mask[:,:,0] > 0.5).astype('int')

    # Create a new figure for each image
    plt.figure(figsize=(15, 5))

    # Plot original image
    plt.subplot(1, 3, 1)
    plt.imshow(image)
    plt.title("Original Image")
    plt.axis('off')

    # Plot predicted mask
    plt.subplot(1, 3, 2)
    plt.imshow(pred_mask.squeeze())
    plt.title("Predicted Mask")
    plt.axis('off')

    # Plot post-processed mask
    plt.subplot(1, 3, 3)
    plt.imshow(post_process)
    plt.title("Post-Processed Mask")
    plt.axis('off')

    # Show the figure
    plt.tight_layout()
    plt.show()


In [ ]:

images, filenames = load_images_from_folder(folder_path)

# Create the mask_output folder if it doesn't exist
mask_output_folder = "/kaggle/working/mask_output"
os.makedirs(mask_output_folder, exist_ok=True)

# Iterate through images and filenames
for image, filename in zip(images, filenames):
    pred_mask = model.predict(expand_dims(image, axis=0))[0]
    post_process = (pred_mask[:,:,0] > 0.5).astype('int')

    # Save post-processed mask
    mask_filename = os.path.splitext(filename)[0] + "_mask.png"  # Append "_mask" to original filename
    mask_filepath = os.path.join(mask_output_folder, mask_filename)
    cv2.imwrite(mask_filepath, post_process * 255)  # Save the mask as a binary image


In [ ]:
# only process 3 images

import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import expand_dims
from tensorflow.image import resize

# Path to the folder containing images
folder_path = "/kaggle/working/images"

# Function to load and preprocess images from a folder
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        if img is not None:
            # Resize to 256x256
            img = resize(img, (256, 256))
            # Normalize
            img = img / 255.0
            images.append(img)
    return images

# Load images from the folder
images = load_images_from_folder(folder_path)


# Only process the first 3 images
images = images[:3]


# Iterate through images
for image in images:
    pred_mask = model.predict(expand_dims(image, axis=0))[0]
    post_process = (pred_mask[:,:,0] > 0.5).astype('int')

    # Create a new figure for each image
    plt.figure(figsize=(15, 5))

    # Plot original image
    plt.subplot(1, 3, 1)
    plt.imshow(image)
    plt.title("Original Image")
    plt.axis('off')

    # Plot predicted mask
    plt.subplot(1, 3, 2)
    plt.imshow(pred_mask.squeeze())
    plt.title("Predicted Mask")
    plt.axis('off')

    # Plot post-processed mask
    plt.subplot(1, 3, 3)
    plt.imshow(post_process)
    plt.title("Post-Processed Mask")
    plt.axis('off')

    # Show the figure
    plt.tight_layout()
    plt.show()


In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import expand_dims
from tensorflow.image import resize

# Path to the folder containing images
folder_path = "/kaggle/working/images"

# Function to load and preprocess images from a folder
def load_images_from_folder(folder):
    images = []
    filenames = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        if img is not None:
            # Resize to 256x256
            img = resize(img, (256, 256))
            # Normalize
            img = img / 255.0
            images.append(img)
            filenames.append(filename)
    return images, filenames

# Load images from the folder
images, filenames = load_images_from_folder(folder_path)

# Create the mask_output folder if it doesn't exist
mask_output_folder = "/kaggle/working/mask_output"
os.makedirs(mask_output_folder, exist_ok=True)

# Iterate through images and filenames
for image, filename in zip(images, filenames):
    pred_mask = model.predict(expand_dims(image, axis=0))[0]
    post_process = (pred_mask[:,:,0] > 0.5).astype('int')

    # Save post-processed mask
    mask_filename = os.path.splitext(filename)[0] + "_mask.png"  # Append "_mask" to original filename
    mask_filepath = os.path.join(mask_output_folder, mask_filename)
    cv2.imwrite(mask_filepath, post_process * 255)  # Save the mask as a binary image


In [ ]:
# Final Code to process large images, splits into 1000 * 1000 then resize to 256

import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import expand_dims
from tensorflow.image import resize
import tensorflow as tf

# Path to the large image
large_image_path = "/kaggle/input/chicago/41.612898753320216 -87.30889155863774_Chicago_pilot_1.tif"

# Path to save predicted masks
output_folder = "/kaggle/working/"

# Function to predict masks for large image patches
def predict_masks(image, model):
    # Define patch size
    patch_size = (1000, 1000)
    
    # Calculate number of patches in each dimension
    num_patches_h = image.shape[0] // patch_size[0] + 1
    num_patches_w = image.shape[1] // patch_size[1] + 1
    
    # Initialize empty array to store predicted masks
    predicted_masks = np.zeros(image.shape[:2], dtype=np.float32)
    
    # Iterate through patches
    for i in range(num_patches_h):
        for j in range(num_patches_w):
            # Calculate patch coordinates
            start_h = i * patch_size[0]
            end_h = min(start_h + patch_size[0], image.shape[0])
            start_w = j * patch_size[1]
            end_w = min(start_w + patch_size[1], image.shape[1])
            
            # Extract patch
            patch = image[start_h:end_h, start_w:end_w]
            
            # Resize and normalize patch
            patch = resize(patch, (256, 256)) / 255.0
            
            # Predict mask for patch
            pred_mask = model.predict(expand_dims(patch, axis=0))[0]
            post_process = (pred_mask[:,:,0] > 0.8).astype('float32')
            
            # Expand dimensions to make it 3D
            post_process = tf.expand_dims(post_process, axis=-1)
            
            # Resize the post_process array to match patch size
            post_process = tf.image.resize(post_process, [end_h - start_h, end_w - start_w])
            
            # Remove dimensions of size 1 from the shape of the tensor
            post_process = tf.squeeze(post_process)
            
            # Place predicted patch into correct location in the array
            predicted_masks[start_h:end_h, start_w:end_w] = post_process.numpy()
            
    return predicted_masks


# Load the large image
large_image = cv2.imread(large_image_path)
large_image_rgb = cv2.cvtColor(large_image, cv2.COLOR_BGR2RGB)

# Predict masks for the large image
predicted_mask = predict_masks(large_image_rgb, model)

# Save predicted mask
os.makedirs(output_folder, exist_ok=True)
output_path = os.path.join(output_folder, "patch1000 sen 8 41.612898753320216 -87.30889155863774_Chicago_pilot_1_predicted_mask.png")
plt.imsave(output_path, predicted_mask, cmap='gray')

print("Prediction saved at:", output_path)


In [ ]:
#Code to process large images, splits into 1000 * 1000 then resize to 256

import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import expand_dims
from tensorflow.image import resize
import tensorflow as tf

# Path to the large image
large_image_path = "/kaggle/input/chicago/41.612898753320216 -87.30889155863774_Chicago_pilot_1.tif"

# Path to save predicted masks
output_folder = "/kaggle/working/"

# Function to predict masks for large image patches
def predict_masks(image, model):
    # Define patch size
    patch_size = (1000, 1000)
    
    # Calculate number of patches in each dimension
    num_patches_h = image.shape[0] // patch_size[0] + 1
    num_patches_w = image.shape[1] // patch_size[1] + 1
    
    # Initialize empty array to store predicted masks
    predicted_masks = np.zeros(image.shape[:2], dtype=np.float32)
    
    # Iterate through patches
    for i in range(num_patches_h):
        for j in range(num_patches_w):
            # Calculate patch coordinates
            start_h = i * patch_size[0]
            end_h = min(start_h + patch_size[0], image.shape[0])
            start_w = j * patch_size[1]
            end_w = min(start_w + patch_size[1], image.shape[1])
            
            # Extract patch
            patch = image[start_h:end_h, start_w:end_w]
            
            # Resize and normalize patch
            patch = resize(patch, (256, 256)) / 255.0
            
            # Predict mask for patch
            pred_mask = model.predict(expand_dims(patch, axis=0))[0]
            post_process = (pred_mask[:,:,0] > 0.5).astype('float32')
            
            # Expand dimensions to make it 3D
            post_process = tf.expand_dims(post_process, axis=-1)
            
            # Resize the post_process array to match patch size
            post_process = tf.image.resize(post_process, [end_h - start_h, end_w - start_w])
            
            # Remove dimensions of size 1 from the shape of the tensor
            post_process = tf.squeeze(post_process)
            
            # Place predicted patch into correct location in the array
            predicted_masks[start_h:end_h, start_w:end_w] = post_process.numpy()
            
    return predicted_masks


# Load the large image
large_image = cv2.imread(large_image_path)
large_image_rgb = cv2.cvtColor(large_image, cv2.COLOR_BGR2RGB)

# Predict masks for the large image
predicted_mask = predict_masks(large_image_rgb, model)

# Save predicted mask
os.makedirs(output_folder, exist_ok=True)
output_path = os.path.join(output_folder, "patch100 sen 5 41.612898753320216 -87.30889155863774_Chicago_pilot_1_predicted_mask.png")
plt.imsave(output_path, predicted_mask, cmap='gray')

print("Prediction saved at:", output_path)


In [ ]:
#Code to process large images, but splits into 255's

import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import expand_dims
from tensorflow.image import resize

# Path to the large image
large_image_path = "/kaggle/input/chicago/41.612898753320216 -87.30889155863774_Chicago_pilot_1.tif"

# Path to save predicted masks
output_folder = "/kaggle/working/"

# Function to predict masks for large image patches
def predict_masks(image, model):
    # Define patch size
    patch_size = (256, 256)
    
    # Calculate number of patches in each dimension
    num_patches_h = image.shape[0] // patch_size[0] + 1
    num_patches_w = image.shape[1] // patch_size[1] + 1
    
    # Initialize empty array to store predicted masks
    predicted_masks = np.zeros(image.shape[:2], dtype=np.float32)
    
    # Iterate through patches
    for i in range(num_patches_h):
        for j in range(num_patches_w):
            # Calculate patch coordinates
            start_h = i * patch_size[0]
            end_h = min(start_h + patch_size[0], image.shape[0])
            start_w = j * patch_size[1]
            end_w = min(start_w + patch_size[1], image.shape[1])
            
            # Extract patch
            patch = image[start_h:end_h, start_w:end_w]
            
            # Resize and normalize patch
            patch = resize(patch, (256, 256)) / 255.0
            
            # Predict mask for patch
            pred_mask = model.predict(expand_dims(patch, axis=0))[0]
            post_process = (pred_mask[:,:,0] > 0.5).astype('float32')
            
            # Place predicted patch into correct location in the array
            predicted_masks[start_h:end_h, start_w:end_w] = post_process[:end_h-start_h, :end_w-start_w]
            
    return predicted_masks


# Load the large image
large_image = cv2.imread(large_image_path)
large_image_rgb = cv2.cvtColor(large_image, cv2.COLOR_BGR2RGB)

# Predict masks for the large image
predicted_mask = predict_masks(large_image_rgb, model)

# Save predicted mask
os.makedirs(output_folder, exist_ok=True)
output_path = os.path.join(output_folder, "41.612898753320216 -87.30889155863774_Chicago_pilot_1_predicted_mask.png")
plt.imsave(output_path, predicted_mask, cmap='gray')

print("Prediction saved at:", output_path)
